In [5]:
# pip install langchain_chroma

In [6]:
# building a sample vector db
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [7]:
loader= TextLoader("speech.txt")
docs= loader.load()
docs

[Document(metadata={'source': 'speech.txt'}, page_content="Life has a beautiful way of surprising us when we least expect it. In moments of difficulty, even a small gesture of kindness can bring immense comfort. Whether it's a warm smile, a thoughtful message, or simply someone showing up when you're down, these acts remind us that we are not alone. Healing isn't always about medicine—sometimes, it's about presence. The care we receive during our weakest times becomes a memory we carry forever. Gratitude grows quietly but deeply in such moments. And sometimes, just one person, arriving with love and sincerity, can make all the difference in the world.\n\n")]

In [8]:
# split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

In [10]:
embeddings=OllamaEmbeddings(model="gemma:2b")
vector_db = Chroma.from_documents(texts, embeddings)
vector_db

In [11]:
# query it
query = "What is the main topic of the speech?"
results = vector_db.similarity_search(query)
results[0].page_content

"Life has a beautiful way of surprising us when we least expect it. In moments of difficulty, even a small gesture of kindness can bring immense comfort. Whether it's a warm smile, a thoughtful message, or simply someone showing up when you're down, these acts remind us that we are not alone. Healing isn't always about medicine—sometimes, it's about presence. The care we receive during our weakest times becomes a memory we carry forever. Gratitude grows quietly but deeply in such moments. And sometimes, just one person, arriving with love and sincerity, can make all the difference in the world."

In [12]:
# saving the database
vector_db=Chroma.from_documents(texts, embeddings,persist_directory="./chroma_db")

In [13]:
# loading the database from disk
db2 = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
dox=db2.similarity_search(query)
dox

[Document(id='088c6f5c-db49-486c-8a90-f101a97ffbe3', metadata={'source': 'speech.txt'}, page_content="Life has a beautiful way of surprising us when we least expect it. In moments of difficulty, even a small gesture of kindness can bring immense comfort. Whether it's a warm smile, a thoughtful message, or simply someone showing up when you're down, these acts remind us that we are not alone. Healing isn't always about medicine—sometimes, it's about presence. The care we receive during our weakest times becomes a memory we carry forever. Gratitude grows quietly but deeply in such moments. And sometimes, just one person, arriving with love and sincerity, can make all the difference in the world.")]

In [14]:
# retrieving the original document
ret=vector_db.as_retriever()
ret.invoke(query)[0].page_content

"Life has a beautiful way of surprising us when we least expect it. In moments of difficulty, even a small gesture of kindness can bring immense comfort. Whether it's a warm smile, a thoughtful message, or simply someone showing up when you're down, these acts remind us that we are not alone. Healing isn't always about medicine—sometimes, it's about presence. The care we receive during our weakest times becomes a memory we carry forever. Gratitude grows quietly but deeply in such moments. And sometimes, just one person, arriving with love and sincerity, can make all the difference in the world."